In [1]:
# Packages needed, may not be in git yet

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.constants as sc
%matplotlib inline
from eos import *
#from astro_const import *
from structure_template import *
from reactions import *
from scipy.optimize import bisect
from astro_const import *

In [2]:
delta_m = 1e-4
eta = 1e-11
xi = 0.01
mue = 2
masses = Msun*np.arange(0.1,1.1,0.1)
Pcs = np.zeros(10)
Pguesses = np.zeros(10)

def f(Pc,Mwant,delta_m,eta,xi,mue):
    m,r,p = integrate(Pc,delta_m,eta,xi,mue)
    M = m[-1]
    return(M-Mwant)

for i in range(len(masses)):
    Pguess = pressure_guess(masses[i],mue)
    Pguesses[i] = Pguess
    Pc = bisect(f,Pguess/100,100*Pguess,args = (masses[i],delta_m,eta,xi,mue))
    Pcs[i] = Pc

central_dens = density(Pcs,mue)

In [6]:
Radii = np.zeros(10)

for i in range(len(masses)):
    m,r,p = integrate(Pcs[i],delta_m,eta,xi,mue)
    Radii[i] = r[-1]

In [7]:
R = Radii/Rsun
Pcsnorm = Pcs/(G*(masses**2)*Radii**(-4))
central_dens_norm = central_dens/((3*masses)/(4*np.pi*Radii**(3)))

In [8]:
array = np.array([masses/Msun,R,Pcs,Pcsnorm,central_dens,central_dens_norm])
MRdf = pd.DataFrame(array,index = ["$\dfrac{M}{M_{\odot}}$","$\dfrac{R}{R_\odot}$",
                                   "$P_c$","$\dfrac{P_c}{GM^2R^{-4}}$","$\rho_c$",
                                   "$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$"]).transpose()
MRdf.columns = ["$$\dfrac{M}{M_{\odot}}$$","$$\dfrac{R}{R_\odot}$$",
                                   "$$P_c$$","$$\dfrac{P_c}{GM^2R^{-4}}$$","$$\rho_c$$",
                                   "$$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$$"]
MRdf

,$$\dfrac{M}{M_{\odot}}$$,$$\dfrac{R}{R_\odot}$$,$$P_c$$,$$\dfrac{P_c}{GM^2R^{-4}}$$,$$\rho_c$$,$$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$$
0,0.1,0.027494,1.517984e+19,0.769975,4.063077e+07,5.989740
1,0.2,0.021822,1.530032e+20,0.769975,1.625231e+08,5.989738
2,0.3,0.019063,5.911141e+20,0.769975,3.656769e+08,5.989740
3,0.4,0.017320,1.542176e+21,0.769975,6.500922e+08,5.989741
4,0.5,0.016078,3.244645e+21,0.769975,1.015769e+09,5.989738
5,0.6,0.015130,5.958057e+21,0.769975,1.462708e+09,5.989738
6,0.7,0.014373,9.960030e+21,0.769975,1.990907e+09,5.989741
7,0.8,0.013747,1.554416e+22,0.769975,2.600369e+09,5.989739
8,0.9,0.013218,2.301842e+22,0.769975,3.291092e+09,5.989739
9,1.0,0.012761,3.270397e+22,0.769975,4.063077e+09,5.989741


In [10]:
for i in range(len(masses)):
    Pguess = pressure_guess(masses[i],mue)
    Pc = bisect(f,0.9*Pguess,1.1*Pguess,args = (masses[i],delta_m,eta,xi,mue))
    Pcs[i] = Pc

central_dens = density(Pcs,mue,K = 0.5)

Radii = np.zeros(10)

for i in range(len(masses)):
    m,r,p = integrate(Pcs[i],delta_m,eta,xi,mue)
    Radii[i] = r[-1]

In [11]:
array = np.array([masses/Msun,R,Pcs,Pcsnorm,central_dens,central_dens_norm])
MRdf = pd.DataFrame(array,index = ["$\dfrac{M}{M_{\odot}}$","$\dfrac{R}{R_\odot}$",
                                   "$P_c$","$\dfrac{P_c}{GM^2R^{-4}}$","$\rho_c$",
                                   "$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$"]).transpose()
MRdf.columns = ["$$\dfrac{M}{M_{\odot}}$$","$$\dfrac{R}{R_\odot}$$",
                                   "$$P_c$$","$$\dfrac{P_c}{GM^2R^{-4}}$$","$$\rho_c$$",
                                   "$$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$$"]
MRdf

,$$\dfrac{M}{M_{\odot}}$$,$$\dfrac{R}{R_\odot}$$,$$P_c$$,$$\dfrac{P_c}{GM^2R^{-4}}$$,$$\rho_c$$,$$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$$
0,0.1,0.027494,1.517984e+19,0.769975,6.158472e+07,5.989740
1,0.2,0.021822,1.530032e+20,0.769975,2.463389e+08,5.989738
2,0.3,0.019063,5.911141e+20,0.769975,5.542625e+08,5.989740
3,0.4,0.017320,1.542176e+21,0.769975,9.853556e+08,5.989741
4,0.5,0.016078,3.244645e+21,0.769975,1.539618e+09,5.989738
5,0.6,0.015130,5.958057e+21,0.769975,2.217050e+09,5.989738
6,0.7,0.014373,9.960030e+21,0.769975,3.017651e+09,5.989741
7,0.8,0.013747,1.554416e+22,0.769975,3.941422e+09,5.989739
8,0.9,0.013218,2.301842e+22,0.769975,4.988363e+09,5.989739
9,1.0,0.012761,3.270397e+22,0.769975,6.158472e+09,5.989741


In [12]:
for i in range(len(masses)):
    Pguess = pressure_guess(masses[i],mue)
    Pc = bisect(f,0.9*Pguess,1.1*Pguess,args = (masses[i],delta_m,eta,xi,mue))
    Pcs[i] = Pc

central_dens = density(Pcs,mue,K = 2)

Radii = np.zeros(10)

for i in range(len(masses)):
    m,r,p = integrate(Pcs[i],delta_m,eta,xi,mue)
    Radii[i] = r[-1]

In [13]:
array = np.array([masses/Msun,R,Pcs,Pcsnorm,central_dens,central_dens_norm])
MRdf = pd.DataFrame(array,index = ["$\dfrac{M}{M_{\odot}}$","$\dfrac{R}{R_\odot}$",
                                   "$P_c$","$\dfrac{P_c}{GM^2R^{-4}}$","$\rho_c$",
                                   "$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$"]).transpose()
MRdf.columns = ["$$\dfrac{M}{M_{\odot}}$$","$$\dfrac{R}{R_\odot}$$",
                                   "$$P_c$$","$$\dfrac{P_c}{GM^2R^{-4}}$$","$$\rho_c$$",
                                   "$$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$$"]
MRdf

,$$\dfrac{M}{M_{\odot}}$$,$$\dfrac{R}{R_\odot}$$,$$P_c$$,$$\dfrac{P_c}{GM^2R^{-4}}$$,$$\rho_c$$,$$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$$
0,0.1,0.027494,1.517984e+19,0.769975,2.680631e+07,5.989740
1,0.2,0.021822,1.530032e+20,0.769975,1.072252e+08,5.989738
2,0.3,0.019063,5.911141e+20,0.769975,2.412568e+08,5.989740
3,0.4,0.017320,1.542176e+21,0.769975,4.289009e+08,5.989741
4,0.5,0.016078,3.244645e+21,0.769975,6.701577e+08,5.989738
5,0.6,0.015130,5.958057e+21,0.769975,9.650271e+08,5.989738
6,0.7,0.014373,9.960030e+21,0.769975,1.313509e+09,5.989741
7,0.8,0.013747,1.554416e+22,0.769975,1.715604e+09,5.989739
8,0.9,0.013218,2.301842e+22,0.769975,2.171311e+09,5.989739
9,1.0,0.012761,3.270397e+22,0.769975,2.680631e+09,5.989741
